In [ ]:
from IPython.display import Image

# TP01: Archivos netCDF Image

Lo primero que vamos a hacer es descargar los datos de marine-copernicus (https://data.marine.copernicus.eu/products), para esto vamos a presentar tres formas posibles de hacerlo:  
- Descarga desde la web.  
- Descarga mediante conexión FTP.  
- Descarga mediante xarray.  

Existe una cuarta variante usando la api MOTU, la cual (me) presenta ciertos problemas con Windows.

## WEB copernicus

Lo primero que vamos a hacer es acceder a la página de marine-copernicus:  
https://data.marine.copernicus.eu/products

una vez ahí nos deberia aparecer en imagen los productos disponibles y un conjunto de filtros que nos va permitir encontrar de forma mas simple el producto (o los productos) deseados.


In [ ]:
print("Figure 1: Visualización marine-copernicus. A la izquierda se pueden observar \nlos filtros, mientras en el centro o a la derecha de los filtros se observan \nprevisualizaciones de los productos.  ")
Image(filename="./images/copernicus-productos.jpeg", width=700, height=None)


Dentro de las opciones del filtro existen diversas variable principla, nivel procesamiento, area, resolución tempora, etc. En parcicular, para encontrar el producto que nos piden en el enunciado basta elegir alguno de estos:

- Main Variable: Temperature o ICE.
- Area: Globan Ocean.
- Temporal Resolution: Daily. 
- Processing Level: L4.

In [ ]:
print("Figure 2: Previsualización del producto Global SST and Sea Ice REP, L4 OSTIA, 0.05deg daily \n(METOFFICE-GLO-SST-L4-RAN-OBS-SST)")
Image(filename="./images/producto.jpeg", width=400, height=100)

El link del producto es:  
https://data.marine.copernicus.eu/product/SST_GLO_SST_L4_REP_OBSERVATIONS_010_011/description


In [ ]:
print("Figure 3: Pantalla del producto. Resaltado en un círculo en rojo la opción de explorar \npara poder descargar los datos de interes.")
Image(filename="./images/descargar-producto.png", width=700, height=100)

In [ ]:
print("Figure 4: Panel inicial. Resaltado en un círculo rojo la opción \npara poder descargar los datos de interes.")
Image(filename="./images/descargar-boton.png", width=300, height=100)

In [ ]:
print("Figure 5: Panel extendido. Resaltado en un círculo verde las opciones para elegir una region en el mapa. \nResaltado en un círculo rojo el boto para descargar, donde adenasm se observa el peso del archivo a descargar.")
Image(filename="./images/descargar-panel.png", width=300, height=100)

## FTP 

Otra  forma para descar los datos es conectarse mediante FTP. Esta te va a parmitir descargar los archivos "crudos", es decir, sin una preselección de lat/lon o que datos se quieren o no. Hay en ciertos sistemas de copernicus donde solo se pueden acceder a datos raw, para estos casos conectarse mediante FTP es una buena opción para automatizar la descarga. 

Importante! Esto puede hacerse desde la terminal sin necesidad de pasar por python.

##### Librerias

Vamos a necesitar una única librería (la cual suele venir por defecto)  
`ftplib: pip install ftplib`


### Código

Vamos a definir dos funciones que nos van a simplificiar y ordenar la conexión y descarga:

**ftp_copernicus**   
función que nos abre la conexión ftp con el servidor. 

```
def ftp_copernicus(host, urs, pwd):
    ftp = ftplib.FTP(host, user, pwd)  
    ftp.encoding = "utf-8"
    return ftp
```

**ftp_download**  
función que nos descarga el archivo. Los parámetros son la conexión ftp, el path donde queremos que guarde el archivo (incluir el no,bre con el que se quiere que se guarde) y el path del servidor donde se encuentra el archivo. 


```
def ftp_download(ftp, localpath, ftppath):
    file = open(localpath, "wb")
    ftp.retrbinary("RETR " + ftppath, file.write)
    file.close()
```


### Usando el código

Lo primero que tenemos que hacer es conectarnos remotamente. Para esto es importante saber que marinie copernicus tiene  dos hosts distintos, a saber ('nrt.cmems-du.eu', 'my.cmems-du.eu') y nos podemos conectar a ambos mediante ftp. 

Para iniciar sessión vamos hacer lo siguiente (lo indicado entre llaves son valores a rellenar):

```
import ftplib
  
user = {my_user_copernicus}  
pwd = {my_pasword_copernicus}  
hosts = ['nrt.cmems-du.eu', 'my.cmems-du.eu']   
copernicus = ftp_copernicus(hosts[0], user, pwd)
```
  
  
**en el interior del servidor**  

Para moverse dentro de una conexión ftp existen dos funciones `cwd` y `nlst`, análogas  a moverse en la terminal. `cwd` nos mueve entre carpetas (incluyendo `cwd("..")` para ir un directorio atras) y `nlst` nos devuelve la lista de archivos/directorios en la ubicación dode estamos.

La estructura interna de ambos host suele ser:  

`"CORE/{PRODUCT_ID}/{DATASET_ID}/{YEAR}/{MONTH}/*.nc"`

CORE es la carpeta principal, de alli uno puede moverse hacia cualquier dataset con la funcion `cwd`.

Por ejemplo:

```
copernicus.cwd("CORE/{PRODUCT_ID}/{DATASET_ID}/{YEAR}/{MONTH}")
listfiles = copernicus.nlst()
```

nos va a guardar en  listfiles la lista de carpetas/archivos existentes en la carpeta del path indicado.

**descarga**  
Para descargar el archivo basta llamar a la función definida  anteriormente:

```
fpt_download(copernicus, {localpath}, {ftppath})
```

##### close  
Siempre cerrar la conexion!!!

```
copernicus.close()
```

## XARRAY 

Otra forma de obtener los datos es usando la libreria xarray.  En este caso vamos a necesitar iniciar sesión en marine-copernicus y la url del dataset. Como en FTP, no todo dataset esta disponible para descargarlo con este protocolo.


**Librerias**  
Vamos a necesitar dos librerías:  
- pydap: pip install pydap
- xarray: pip install xarray as xr


### Código

Vamos a iniciar importando las  librerias:

```
import pydap  
import xarray as xr  
from pydap.client import open_url  
from pydap.cas.get_cookies import setup_session
```

y vamos a definir dos funciones:

**session_copernicus**  
Funcion que nos devuelve una sesión de Marine-Copernicus.

```
def session_copernicus(usr=None, pwd=None):  
    cas_url = 'https://cmems-cas.cls.fr/cas/login'  
    session = setup_session(cas_url, usr, pwd)  
    session.cookies.set("CASTGC", session.cookies.get_dict()['CASTGC'])  
    return sesssion
```

**get_dataset_store**  
Funcion que devuelve con un conector (store) con el dataset. La función prueba primero con un host y, en  caso de error, con el otro.

```
def get_dataset_store(session, datased_id):
    database = ['my', 'nrt']
    url = f'https://{database[0]}.cmems-du.eu/thredds/dodsC/{datasetID}'
    try:
        data_store = xr.backends.PydapDataStore(open_url(url, session=session))
    except:
        url = f'https://{database[1]}.cmems-du.eu/thredds/dodsC/{datasetID}'
        data_store = xr.backends.PydapDataStore(open_url(url, session=session))
    return  data_store
```


### Usando el código  


Definimos primero variables de configuración, es decir, usuario y  contraseña de marine-copernicus y el id del dataset que vamos a querer:
```
DATASETID = "METOFFICE-GLO-SST-L4-REP-OBS-SST"
USERNAME = "{}"
PASSWORD = "{}"
```

Con esto, simplemente basta ejecutar las siguientes lineas: 

```
session = session_copernicus(usr=USERNAME, pwd=PASSWORD)
data_store  = get_dataset_store(session, dataset_id)
DS = xr.open_dataset(data_store)
```

En una notebook (de jupyternotebook o colab) se visualizá como la siguiente tabla:

In [ ]:
DS

El beneficio de este format, es que nos permite acecder una time/lat/lon con sencillez:

```
LATITUDE = slice(35, 60)
LONGITUDE = slice(-15, 5)
TIME = '2022-01-01'
ds = DS.sel(time=TIME, lat=LATITUDE, lon=LONGITUDE)
```

Importante! Los nombres de los valores de las dimensiones van a depender del dataset.

##### FUENTE
 https://help.marine.copernicus.eu/en/articles/4854800-how-to-open-and-visualize-copernicus-marine-data-using-python